<a href="https://colab.research.google.com/github/YuriyKozhubaev/PY100/blob/main/PY110_lecture_2_1_Decorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Декораторы

## Замыкание функций
Разберем, что такое функции высших порядков.

**Функция высшего порядка** — в программировании функция, принимающая в  
качестве аргументов другие функции или возвращающая другую функцию  
в качестве результата.  
Основная идея состоит в том, что функции имеют тот же статус,  
что и другие объекты данных.  

Мы можем передавать функции как параметры...
```python
def my_func(inside_func):
    ...
    inside_func()  # Вызов функции принятой в качестве аргумента
    ...
```

... можем возвращать как результат ...
```python
def a():
    def b(): 
        pass
    return b
```
... и присваивать их другим переменным
```python
def a(): 
    pass

b = a
b()
```

Сделаем функцию, которая будет выполнять принимаемую функцию дважды

In [ ]:
def twice_func(inside_func):
    inside_func()
    inside_func()

def hello():
    print("Hello")
    
test = twice_func(hello)

Hello
Hello


In [ ]:
test()

TypeError: ignored

Видим что на момент инициализации `twice_func`  
сразу происходит выполнение функци `hello`.  

Давайте подумаем как можно решить этот вопрос, чтобы мы могли присвоить  
результат переменной `test` и после вызывать нашу функцию,  
которую мы передаем в качестве аргумента?

Теперь перейдем к замыканию функций. Разберем что это такое.  

**Замыкание** в программировании — функция, в теле которой присутствуют  
ссылки на переменные, объявленные вне тела этой функции в окружающем коде  
и не являющиеся её параметрами.  
Не путайте это с глобальными переменными, с котороми может работать функция.

In [ ]:
CONST_PI = 3.14

def square(r: float) -> float:
    return CONST_PI * r ** 2  # это не замыкание функции

In [ ]:
def hello():
    welcome_phrase = "Hello"  # nonlocal переменная
    def welcome_print(name: str):
        local_var = "локальная переменная"
        print(f"{welcome_phrase}, {name}")
    return welcome_print

inside_func = hello()
print(inside_func)

<function hello.<locals>.welcome_print at 0x7fc8e4b36d40>


In [ ]:
inside_func_second = hello()
print(inside_func_second)

<function hello.<locals>.welcome_print at 0x7fc8e3a5dcb0>


In [ ]:
inside_func(name="World")

Hello, World


In [ ]:
inside_func(name="Test")

Hello, Test


Таких уровней вложенности может быть сколько угодно

In [ ]:
def hello():
    welcome_phrase = "Hello"  # nonlocal переменная
    def welcome_print(name: str):
        print(f"{welcome_phrase}, {name}")
        def tmp():
            welcome_phrase + name
    return welcome_print

In [ ]:
count = 0  # глобальная
def wrapper():
    count = count + 1
    print(count)

wrapper()

UnboundLocalError: ignored

In [ ]:
count = 50  # глобальная
def wrapper():
    count =+ 10
    print(count)

wrapper()

10


In [ ]:
def counter():
    count = 0  # nonlocal переменная wrapper,  counter - локальная
    def wrapper():
        count = count + 1
        print(count)
    return wrapper

inside_func = counter()
print(inside_func)

<function counter.<locals>.wrapper at 0x7fc8dff69b90>


In [ ]:
inside_func()  # UnboundLocalError: local variable 'count' referenced before assignment

UnboundLocalError: ignored

Решается этот момент использование ключевого слова `nonlocal`.  
Такой механизм использования поддерживается и используется  
в практических задачах, в отличии от ключевого слова `global`.

In [ ]:
def counter():
    count = 0  # nonlocal переменная
    def wrapper():
        ...
        nonlocal count
        count += 1
        print(count)
    return wrapper

inside_func = counter()
print(inside_func)

<function counter.<locals>.wrapper at 0x7fc8dff698c0>


In [ ]:
inside_func()
inside_func()

1
2


In [ ]:
# новая функция со своим счетчиком
new_inside = counter()
new_inside()

1


In [ ]:
inside_func()

3


Возникает справедливый вопрос, а зачем всё это???  

Рассмотрим такую задачу, я хочу написать функцию,  
которая будет всегда возвращать аргумент по одну и тому же индексу.

```python
def get_item(sequence):
    index = 1
    return sequence[index]
```

```python
def add(sequence, index=1):
    return sequence[index]

add([1, 2, 3])  # 2
add([1, 2, 3], 2)  # 3
```

In [ ]:
# Реализация с помощью именованных функций:
def itemgetter(index):
    def _itemgetter(sequence):
        # захват переменной "index" из nonlocal области видимости
        return sequence[index]
    return _itemgetter

# функция, которая будет всегда возвращать второй элемент 
itemgetter_1 = itemgetter(1)
print(itemgetter_1)

<function itemgetter.<locals>._itemgetter at 0x7fc8dff69dd0>


In [ ]:
print(itemgetter_1([1, 2, 3]))
print(itemgetter_1([5, 7, 9]))

2
7


In [ ]:
# функция, которая будет всегда возвращать первый элемент 
itemgetter_0 = itemgetter(0)

print(itemgetter_0("abc"))
print(itemgetter_0("zxy"))  

a
z


In [ ]:
dict([('apple', 3), ('banana', 2), ('pear', 5), ('orange', 1)])

{'apple': 3, 'banana': 2, 'orange': 1, 'pear': 5}

In [ ]:
dict_ = {
    'apple': 3, 
    'banana': 2, 
    'orange': 1, 
    'pear': 5
}
print(dict(sorted(dict_.items(), key=lambda item: item[1])))

{'orange': 1, 'banana': 2, 'apple': 3, 'pear': 5}


In [ ]:
get_fruit_count = itemgetter(1)
print(dict(sorted(dict_.items(), key=get_fruit_count)))

{'orange': 1, 'banana': 2, 'apple': 3, 'pear': 5}


Но лучше использовать модуль [operator](https://docs.python.org/3/library/operator.html#operator.itemgetter)

# Декораторы
 
Паттерн **декоратор** предназначен для подключения дополнительного поведения  
к основному объекту, которое будет выполняться до, после или вместо  
поведения основного объекта.  

Например, хотим включать таймер до вызова функции и выключать после,  
а потом – выдавать результат.

In [ ]:
def my_decorator(a_function_to_decorate):
    # Здесь мы определяем новую функцию – wrapper («обертку»).  
    # Она нам нужна, чтобы выполнять каждый раз при
    # вызове оригинальной функции, а не только один раз
    def wrapper():
        # здесь поместим код, которые будет выполняться до вызова, потом вызов
        # оригинальной функции, потом код после вызова
        print("Я буду выполнен до основного вызова!")
        
        result = a_function_to_decorate()  # не забываем вернуть значение исходной функции
        
        print("Я буду выполнен после основного вызова!")
        return result
    return wrapper

In [ ]:
def my_function():
    print("Я – оборачиваемая функция!")
    return 0
print(my_function)

my_function = my_decorator(my_function)  # замыкание
print(my_function)

<function my_function at 0x7fa49a687290>
<function my_decorator.<locals>.wrapper at 0x7fa49a670170>


In [ ]:
result = my_function()
print(result)

Я буду выполнен до основного вызова!
Я – оборачиваемая функция!
Я буду выполнен после основного вызова!
0


Что теперь будет в переменной `my_function`?  
Функция `my_function`, декоратор `my_decorator` или может быть `wrapper`?

In [ ]:
print(my_function)

<function my_decorator.<locals>.wrapper at 0x7fa49a670170>


In [ ]:
print(my_function())

Я буду выполнен до основного вызова!
Я – оборачиваемая функция!
Я буду выполнен после основного вызова!
0


И так будет ~~с каждым~~ при каждом вызове,  
а не только с на момент инициализации.  
Одним и тем же декоратором можно обвернуть любую функцию.

---
### Пример декоратора
Давайте реализуем этот паттерн на Python.  
Поведение декоратора должно выполняться каждый раз,  
когда вызывается оригинальная функция.  

Нам понадобится функция, которая будет возвращать функцию,  
внутри которой будет выполняться дополнительное и основное поведение.

In [ ]:
# пример с посчетом времени выполнения функции
import time


def decorator_time(fn):
    def wrapper():
        print(f"Запустилась функция {fn}")
        t0 = time.time()
        result = fn()
        dt = time.time() - t0
        print(f"Функция выполнилась. Время: {dt:.10f}")
        return result
    return wrapper

def pow_2():
    return 10000000 ** 2

def in_build_pow():
    return pow(10000000, 2)


pow_2 = decorator_time(pow_2)
in_build_pow = decorator_time(in_build_pow)

print(pow_2())
print(in_build_pow())

Запустилась функция <function pow_2 at 0x7fa49a5ee830>
Функция выполнилась. Время: 0.0000007153
100000000000000
Запустилась функция <function in_build_pow at 0x7fa49a5eeb90>
Функция выполнилась. Время: 0.0000019073
100000000000000


In [ ]:
def in_build_pow(base):
    return pow(base, 2)

def time(fn):
    print(f"Запустилась функция {fn}")
    t0 = time.time()
    result = fn()
    dt = time.time() - t0
    print(f"Функция выполнилась. Время: {dt:.10f}")
    return result

---
### Синтаксический сахар
Данная конструкция была настолько часто используема в языке,  
что ее оформили в качестве синтаксической конструкции

```python
@my_decorator  # my_function = my_decorator(my_function)
def my_function():
    pass
```

При этом будет происходить все то же самое, аналогичное
```python
my_function = my_decorator(my_function)
```

In [ ]:
# объявляем декоратор
def do_it_twice(func):
    print("Я выполняюсь на момент декорирования")
    def wrapper():
        func()
        func()
    return wrapper

In [ ]:
# декорируем функцию
@do_it_twice  # say_whee = do_it_twice(say_whee)
def say_whee():
    print("Whee!")

Я выполняюсь на момент декорирования


In [ ]:
# вызываем задекорированую функцию
say_whee()

Whee!
Whee!


In [ ]:
# какая функция скрывается под переменной say_whee??
print(say_whee)

<function do_it_twice.<locals>.wrapper at 0x7fa49a5ee170>


---
### Передача аргументов в декорируемую функцию
До этого мы с вами декорировали только функции без параметров.  
А как их передавать?

Попробуем задекорировать функцию, которая умеет принимать параметры

In [ ]:
def do_it_twice(func):
    def wrapper():
        func()
        func()
    return wrapper

@do_it_twice
def say_word(word):
    print(word)

say_word("Oo!!!")  # TypeError: wrapper() takes 0 positional arguments but 1 was given

TypeError: ignored

Напомним, что декораторы – это лишь обертка над
```python
my_fun = my_decorator(my_fun)
```

И каждый следующий вызов `my_fun` – это вызов `wrapper`.  
Следовательно, `wrapper` должен уметь принимать параметры и  
передавать их в вызываемую функцию.

In [ ]:
# декоратор, в котором встроенная функция умеет принимать аргументы
def do_it_twice(func):
    def wrapper(arg):
        func(arg)
        func(arg)
    return wrapper

@do_it_twice
def say_word(word):
    print(word)

say_word("Oo!!!")

Oo!!!
Oo!!!


In [ ]:
say_word("Hello!!")

Hello!!
Hello!!


In [ ]:
@do_it_twice
def two_print(other_word):
    print(other_word)

two_print("rewrwe")

rewrwe
rewrwe


In [ ]:
say_word("Hello!!")
two_print("rewrwe")

Hello!!
Hello!!
rewrwe
rewrwe


### args и kwargs

Оператор `*` распаковывает множественные значения.  


In [ ]:
a, b = 1, 2
print(a)
print(b)

1
2


In [ ]:
a = 1, 2
print(a)

(1, 2)


In [ ]:
tuple_ = 1, 
first_value, *last_values = tuple_
print(first_value)
print(last_values)

(1,)
[]


In [ ]:
tuple_ = 1, 2, 3, 4, 5 
first_value, *last_values = tuple_
print(first_value)
print(last_values)

In [ ]:
tuple_ = 1, 2, 3, 4, 5
*first_values, last_value = tuple_
print(first_values)
print(last_value)

[1, 2, 3, 4]
5


In [ ]:
tuple_ = 1, 2, 3, 4, 5
first_value, *middle_values, last_value = tuple_
print(first_value)
print(middle_values)
print(last_value)

1
[2, 3, 4]
5


In [ ]:
tuple_ = 1, 2, 3, 4, 5
first_value, *middle_values, prev_last_value, last_value = tuple_
print(first_value)
print(middle_values)
print(prev_last_value)
print(last_value)

1
[2, 3]
4
5


In [ ]:
print(middle_values)

[2, 3]


In [ ]:
print(*middle_values)

2 3


In [ ]:
list_ = [1, 2]
first_value, *middle_values, last_value = list_
print(first_value)
print(middle_values)
print(last_value)

1
[]
2


In [ ]:
list_ = [1]
first_value, *middle_values, last_value = list_
print(first_value)
print(middle_values)
print(last_value)

ValueError: ignored

Существует два типа параметров функции

- позиционные - *args — это сокращение от arguments (аргументы)
- именованные - **kwargs — это сокращение от keyword arguments (именованные аргументы)

In [ ]:
def print_args(*args):
    print(type(args), args)

print_args()
print_args(1)
print_args(1, 2, 3)

<class 'tuple'> ()
<class 'tuple'> (1,)
<class 'tuple'> (1, 2, 3)


In [ ]:
def print_kwargs(**kwargs):
    print(type(kwargs), kwargs)

print_kwargs()
print_kwargs(kwarg1="kwarg1")
print_kwargs(kwarg1="kwarg1", kwarg2="kwarg2")

<class 'dict'> {}
<class 'dict'> {'kwarg1': 'kwarg1'}
<class 'dict'> {'kwarg1': 'kwarg1', 'kwarg2': 'kwarg2'}


In [ ]:
def print_args_kwargs(*args, **kwargs):
    for index, arg in enumerate(args):
        print(f"Позиционный аргумент {index}: {arg}")

    for key, kwarg in kwargs.items():
        print(f"Именованный аргумент {key}: {kwarg}")

print_args_kwargs(1, 2, 3, kwarg1="kwarg1", kwarg2="kwarg2")

Позиционный аргумент 0: 1
Позиционный аргумент 1: 2
Позиционный аргумент 2: 3
Именованный аргумент kwarg1: kwarg1
Именованный аргумент kwarg2: kwarg2


Для тех кто захочет подробнее её изучить, можно воспользоваться следующим [материалом](https://habr.com/ru/company/ruvds/blog/482464/)

---
### Передача переменного количества аргументов в декорируемую функцию
Логичный вопрос – а как передавать параметры, если мы не знаем, что будет вызвано?

In [ ]:
def do_it_twice(func):
    def wrapper(arg):
        func(arg)
        func(arg)
    return wrapper

@do_it_twice
def say_word(word):
    print(word)
    
@do_it_twice
def say_two_words(word1, word2):
    print(f"{word1}, {word2}")

In [ ]:
say_word("Hello")

Hello
Hello


In [ ]:
say_two_words("Hello", "World")

TypeError: ignored

Чтобы решить этот вопрос нужно использовать **\*args** и **\*\*kwargs**.

In [ ]:
def do_it_twice(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper

@do_it_twice
def say_word(word):
    print(word)
    
@do_it_twice
def say_two_words(word1, word2):
    print(f"{word1}, {word2}")


say_word("Hello")
say_two_words("Hello", "World")

Hello
Hello
Hello, World
Hello, World


### Порядок декорирования
Порядок декорирования важен!

Т.к. декоратор – синтаксическая обертка, при разном порядке декораторов будут разные результаты обертывания

```python
# Это не одно и тоже
my_fun = wrap1(wrap2(my_fun))
my_fun = wrap2(wrap1(my_fun))
```
Или альтернативный вариант оформления
```python
@wrap1
@wrap2
def my_fun():
    pass

# функция будет иметь другое поведение
@wrap2
@wrap1
def my_fun():
    pass
```

### Итоги
Подведем черту под темой декораторов.

Паттерн декоратор нужен для того, чтобы добавлять  
дополнительное поведение вызываемой функции  

**Основной шаблон выглядит вот так**:
```python
def my_decorator(fn):
    print("Этот код будет выведен в момент декорирования функции")
    def wrapper(*args, **kwargs):
        print('Этот код будет выполняться перед каждым вызовом функции')
        result = fn(*args, **kwargs)
        print('Этот код будет выполняться после каждого вызова функции')
        return result
    return wrapper
```

- Декораторы добавляют дополнительное поведение функции
- Декораторы – вызовы дополнительных функций, поэтому  
они немного замедляют ваш код
- Порядок декорирования важен
- Декоратор вызывается один раз, переданные в декоратор  
параметры изменить больше нельзя.
- Если хотите использовать декораторы, с настраиваемыми  
параметрами используйте "фабрику декораторов" 

### Где используются декораторы?

1. В самом языке есть @classmethod, @staticmethod и @property – подробнее в курсе ООП
2. Модуль functools:
 - @functools.lru_cache
 - @functools.wraps
3. Flask
 - @app.route('/api/v1/vm', methods=['POST'])
4. Django  
5. Pytest  
...

---
### Фабрика декораторов *
А теперь представьте, что мы хотим создать декоратор, который печатает какое-нибудь слово до вызова функции и после.  
А слово хотим задавать в момент декорирования

```python
@printword('Whee!')
def tes(): 
    pass
```

Где можно указать этот параметр?

Давайте возьмем шаблон и попробуем куда-нибудь вставить  этот параметр.
```python
def my_decorator(fn):
    def wrapper(*args, **kwargs):
        result = fn(*args, **kwargs)
        return result
    return wrapper
```

В `wrapper` нельзя – там аргументы вызываемой функции.  
В `my_decorator` нельзя – он принимает на вход единственный аргумент – функцию, которую обертывает.  
Что делать? (подсказка - замыкание функций)

In [ ]:
def my_decorator(fn, arg):
    def wrapper(*args, **kwargs):
        result = fn(*args, **kwargs)
        return result
    return wrapper

Решение – давайте создадим фабрику декораторов – объект, который будет возвращать декоратор.

In [ ]:
def decorator_maker(word):
    def my_decorator(fun):
        def wrapper(*ar, **kw):
            print(word)
            print('---')
            res = fun(*ar, **kw)
            print('---')
            print(word)
            return res
        return wrapper
    return my_decorator

@decorator_maker('Hi!')
def f():
    print('Hello')

f()

In [ ]:
@decorator_maker('ololo')
def f1():
    print('Hello')

f1()

В чем отличие от того, что мы разбирали прежде?  

Следует отличать указание декоратора и вызов декоратора
```python
@my_decor
def f():
    pass
```
не идентично 
```python
@my_decor()
def f()
    pass
```
В первом случае вы используете декоратор, во втором – вызывается «создатель» декораторов, который возвращает декоратор

---
Хотим сделать декоратор, который будет взависимости от указанного языка приветствовать пользователя

In [ ]:
def welcome(language):  # фабрика декораторов
    welcome_word = {"en": "Hello!", 
                    "ru": "Привет!"}
    def decorator(fn):  # декоратор
        def wrapper(*args, **kwargs):  # задекорированная функция
            print(welcome_word[language])  # приветствие в зависимости от указанного языка в фабрике декораторов
            fn(*args, **kwargs)  # исходная функция
        return wrapper
    return decorator



In [ ]:
# welcome('ru')(my_func_ru)()
# welcome -> decorator -> wrapper -> fn
@welcome('ru')
def my_func_ru():
    print("Как дела?")
    
my_func_ru()

In [ ]:
@welcome('en')
def my_func_en():
    print("How are you?")
    
my_func_en()
print(my_func_en)

In [ ]:
@welcome
def my_func():
    print("How are you?")
    
print(my_func)